In [ ]:
import nbformat
from common_nb_preprocessors.metadata_injector import MetaDataInjectorPreprocessor, GlobalMetaDataInjectorPreprocessor

def _make_tiny_nb():
    nb = nbformat.v4.new_notebook()
    nb.cells.append(nbformat.v4.new_code_cell("# hide\nimport os"))
    nb.cells.append(nbformat.v4.new_markdown_cell("# This is the actual title"))
    return nb

nb = _make_tiny_nb()
nb, _ = MetaDataInjectorPreprocessor(strings=("hide",), prefix="#").preprocess(
    nb, None
)
assert nb.cells[0]["source"] == "import os"
assert nb.cells[0]["metadata"]["tags"][0] == "hide"
# check idempotency
nb, _ = MetaDataInjectorPreprocessor(strings=("hide",), prefix="#").preprocess(
    nb, None
)
assert nb.cells[0]["metadata"]["tags"][0] == "hide"

nb = _make_tiny_nb()
nb, _ = MetaDataInjectorPreprocessor(strings=("hide",), prefix="#", remove_line=False).preprocess(
    nb, None
)
assert nb.cells[0]["source"] == "# hide\nimport os"
assert nb.cells[0]["metadata"]["tags"][0] == "hide"

In [ ]:
nb = nbformat.v4.new_notebook()
nb.cells.append(nbformat.v4.new_code_cell("# publish true"))
parsed_nb, _ = GlobalMetaDataInjectorPreprocessor(keys=("publish",)).preprocess(
    nb, None
)
assert parsed_nb["metadata"]["publish"] == "true"

nb.cells.append(nbformat.v4.new_code_cell("# publish false"))
parsed_nb, _ = GlobalMetaDataInjectorPreprocessor(keys=("publish",)).preprocess(
    nb, None
)
# will overwrite if global is defined multiple times.
assert parsed_nb["metadata"]["publish"] == "false"

nb = nbformat.v4.new_notebook()
nb.cells.append(nbformat.v4.new_code_cell("# key=value"))
parsed_nb, _ = GlobalMetaDataInjectorPreprocessor(keys=("key",), delimiter="=").preprocess(
    nb, None
)
assert parsed_nb["metadata"]["key"] == "value"

In [ ]:
import nbformat
from common_nb_preprocessors.metadata_injector import jupyter_book_metadata_injector

nb = nbformat.v4.new_notebook()
nb.cells.append(nbformat.v4.new_code_cell("# hide-input \n#hide-output\t\nimport os"))
nbstr = nbformat.writes(nb)
out_nb = jupyter_book_metadata_injector(nbstr)
assert "hide-input" in out_nb.cells[0]["metadata"]["tags"]
assert "hide-output" in out_nb.cells[0]["metadata"]["tags"]